El Tagging es el etiquetado a un documento en clases como:
* sentimiento
* idioma
* estilo (formal, informal, etc.)
* temas cubiertos
* tendencias politicas

El Tagging tiene algunos componentes como:
* function: Como en la extracción, tagging usa funciones para especificar como el modelo debería etiquetar el documento.
* schema: define como se desea que se etiquete el documento.

# Quickstart

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

Pydantic es una librería para validación de datos en Python. Permite tener un control de que los datos se apeguen a una estructura especificada por el usuario.

Aquí se especifica un modelo de Pydantic con propiedades y sus tipos de datos esperados en el esquema:

In [3]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.
Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
    """
)

In [4]:
class Classification(BaseModel):
    sentimient: str = Field(description="The sentiment of the text")
    aggressiveness : int = Field(description="How aggressive the text is on a scale from 1 to 10")
    language: str = Field(description="The language the text is written in")

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

In [6]:
text_input = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input":text_input})

response = llm.invoke(prompt)

response

Classification(sentimient='positivo', aggressiveness=1, language='español')

Si queremos el diccionario de la salida, se puede llamar a la función .model_dump()

In [7]:
text_input = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": text_input})
response = llm.invoke(prompt)

response.model_dump()

{'sentimient': 'Enojo', 'aggressiveness': 8, 'language': 'Español'}

Se observa que el modelo interpreta y clasifica la información correctamente. Sin embargo, los resultados varian mucho en sentido de que las clasificaciones hechas no siguen una estructura común ("español" vs "Español").

Para solventar esta situación se presenta a continuación un metodo de control más ajustado.

# Finer control

Un control más especifico en el schema permite más control del output del modelo.

Especificamente, se puede definir:
* posibles valores para cada propiedad
* descripciones para estar seguros de que el modelo entiende las propiedades
* propiedades requeridas para ser retornadas

Aquí se declara el schema usando Pydantric usando los aspectos mencionados para tener más control:

In [8]:
class Classification(BaseModel):
    sentiment:str = Field(enum=["happy","neutral","sad","angry"])
    aggressiveness:int = Field(
        description="describes how agressive the statement is, the higher the number the more aggressive",
        enum=[1,2,3,4,5]
    )
    language:str = Field(
        enum=["Spanish","English","French","Chinese","Italian"]
    )

In [9]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.
Only exttract the properties mentioned in the 'Classification' function.

Passage:
{input}
    """
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

Ahora las respuestas deberían estar restringidas a las propiedades que hemos definido.

In [10]:
text_input = "Estoy muy decepcionado de ti. Me gustaría nunca haberte conocido!"
prompt = tagging_prompt.invoke({"input":text_input})

llm.invoke(prompt)

Classification(sentiment='sad', aggressiveness=4, language='Spanish')

In [11]:
text_input = "I love how you are not getting what you want, you deserve all the struggiling you are having."
prompt = tagging_prompt.invoke({"input":text_input})

llm.invoke(prompt)

Classification(sentiment='angry', aggressiveness=4, language='English')

In [12]:
text_input = "Eres mi mejor amigo. But I hate you! - Said in a comic way"
prompt = tagging_prompt.invoke({"input":text_input})

llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=2, language='Spanish')

In [13]:
text_input = "很高兴认识您！"
prompt = tagging_prompt.invoke({"input":text_input})

llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='Chinese')